In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import hour, when, col, date_format, to_timestamp
from pyspark.sql.functions import *

In [ ]:
sc = pyspark.SparkContext(appName="HW3-Q1")
sqlContext = SQLContext(sc) 

In [ ]:
def load_data():
    df = sqlContext.read.option("header",True).csv("yellow_tripdata_2019-01_short.csv")
    return df

In [ ]:
df = load_data()
# df.printSchema()

In [ ]:
def clean_data(df):
    '''
    input: df a dataframe
    output: df a dataframe with the all the original columns
    '''
    
    df = df.withColumn("passenger_count", df["passenger_count"].cast("Int").alias("passenger_count"))
    df = df.withColumn("total_amount", df["total_amount"].cast("float").alias("total_amount"))
    df = df.withColumn("tip_amount", df["tip_amount"].cast("float").alias("tip_amount"))
    df = df.withColumn("trip_distance", df["trip_distance"].cast("float").alias("trip_distance"))
    df = df.withColumn("fare_amount", df["fare_amount"].cast("float").alias("fare_amount"))
    df = df.withColumn("tpep_pickup_datetime", df["tpep_pickup_datetime"].cast("timestamp").alias("tpep_pickup_datetime"))
    df = df.withColumn("tpep_dropoff_datetime", df["tpep_dropoff_datetime"].cast("timestamp").alias("tpep_dropoff_datetime"))
                
    return df

In [ ]:
df = clean_data(df)
# df.select(['passenger_count', 'total_amount', 'tip_amount', 'trip_distance', 'fare_amount', 'tpep_pickup_datetime', 'tpep_pickup_datetime']).printSchema()

In [ ]:
def common_pair(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - PULocationID
            - DOLocationID
            - count
            - trip_rate
            
    trip_rate is the average amount (total_amount) per distance (trip_distance)
    
    '''
    
    sql = """
    select 
        PULocationID, DOLocationID, count(*) as count, sum(total_amount)/sum(trip_distance) as trip_rate
    from 
        df
    group by 
        PULocationID, DOLocationID
    order by
        count(*) desc,
        sum(total_amount)/sum(trip_distance) desc
    limit 10
    """
    
    sqlContext.registerDataFrameAsTable(df, "df")
    df = sqlContext.sql(sql)
    
    
    return df

In [ ]:
def time_of_cheapest_fare(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:export 
            - day_night
            - trip_rate
    
    day_night will have 'Day' or 'Night' based on following conditions:
        - From 9am to 8:59:59pm - Day
        - From 9pm to 8:59:59am - Night
            
    trip_rate is the average amount (total_amount) per distance
    
    '''
    
    sql = """
    with a as
    (
        select 
        case when substring(tpep_dropoff_datetime, 12, 2) between 9 and 21 then 'Day' else 'Night' end as day_night, 
        total_amount,
        trip_distance
        
        from df
        
        )
    select 
        day_night,
        avg(total_amount)/avg(trip_distance) as trip_rate
    from 
        a
    group by 
        day_night
    order by
        avg(total_amount)/avg(trip_distance)
    """
    
    sqlContext.registerDataFrameAsTable(df, "df")
    df = sqlContext.sql(sql)
    
    
    return df

In [ ]:
# time_of_cheapest_fare(df).show()

In [ ]:
def passenger_count_for_most_tip(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - passenger_count
            - tip_percent
            
    trip_percent is the percent of tip out of fare_amount
    
    '''
    sql = """
    with a as (
    select 
        passenger_count,
        avg(fare_amount) as fare_amount,
        avg(tip_amount) as tip_amount
    from 
        df
    where
        fare_amount > 2
        and passenger_count > 0
    group by 
        passenger_count)
        
    select
        passenger_count,
        tip_amount * 100 / fare_amount as tip_percent
    from a
    order by 
        tip_amount * 100 / fare_amount desc;
    """
    
    sqlContext.registerDataFrameAsTable(df, "df")
    df = sqlContext.sql(sql)    

    
    return df

In [ ]:
def day_with_traffic(df):
    '''
    input: df a dataframe
    output: df a dataframe with following columns:
            - day_of_week
            - average_speed
    
    day_of_week should be day of week e.g.) Mon, Tue, Wed, ...
    average_speed (miles/hour) is calculated as distance / time (in hours)
    '''
    sql = """
    with a as
    (
    select 
        substring(date_format(tpep_pickup_datetime, 'EEEE'), 1, 3) as day_of_week,
        trip_distance,
        ((cast(tpep_dropoff_datetime as long) - cast(tpep_pickup_datetime as long))/3600) as hour
    from 
        df
    ),
    
    b as
    (
    select
        day_of_week,
        avg(hour) as hour
    from
        a
    group by
        day_of_week
    ),
    c as
    (
    select
        day_of_week,
        avg(trip_distance) as trip_distance
    from
        a
    group by
        day_of_week
    )
    
    select 
        b.day_of_week,
        c.trip_distance/b.hour as average_speed
    from 
        b
    left join c on b.day_of_week = c.day_of_week
    order by
        c.trip_distance/b.hour,
        b.day_of_week
    """
    
    sqlContext.registerDataFrameAsTable(df, "df")
    df = sqlContext.sql(sql)
    
    
    return df

In [ ]:
# day_with_traffic(df).show()
df.select(['*'])